# WS05: Sparse systems and pivoting

These exercises are indented to give you practice at using the material on numerical approximation and are intended to reinforce the material that was covered in lectures.

Please attempt the worksheet before your tutorial. Support is available in your tutorial or in the Class Team.

## Part a (pen and paper warm up)

### 1. Sparse matrix storage
Write down the three arrays: array of floating point numbers (say A_real) to store the non-zero entries, array of integers (say I_row) to store the row number and array of integers (say I_col) to store the column number.

   $$
   \begin{pmatrix}
   3 & 0 & 1.5 & 0 \\ 2 & 4 & 2.2 & 0 \\ 0 & 2 & 6 & 0 \\ 0 & 0 & 4 & -9
   \end{pmatrix}
   $$


### 2. Gaussian elimination with pivoting
Using GE with pivoting to solve the following linear system of equations
   $$
   \begin{pmatrix}
   0 & 1 & 0 \\ 
   1 & 1.001 & 1 \\ 
   100 & 100 & 0 \\ 
   \end{pmatrix}
   \begin{pmatrix}
   x_1 \\ 
   x_2 \\
   x_3
   \end{pmatrix} =
   \begin{pmatrix}
    0 \\ 1002 \\ 200
   \end{pmatrix}. 
   $$

### 3. Sparse matrix storage and access
To implement a direct solver such as Gaussian elimination, is it convenient to store a matrix as sparse matrix? What's about iterative solver: Jacobi iteration or Gauss-Seidel iteration?


## Part b (code implementations and testing)

### 4. Sparse matrix multiplication

In [1]:
import numpy as np

A_real = np.array([3, 1.5, 2,4 ,2.2, 2, 6, 4, -9], dtype=np.float64)
I_row = np.array([0, 0, 1, 1, 1, 2, 2, 3, 3], dtype=np.int32)
I_col = np.array([0, 2, 0, 1, 2, 1, 2, 2, 3], dtype=np.int32) 

nonzero = len(A_real)
dim = 4
y = np.zeros(dim) + 1.
z = np.zeros(dim)

for k in range(nonzero):
    z[I_row[k]] = z[I_row[k]] + A_real[k] * y[I_col[k]]

print(z)

[ 4.5  8.2  8.  -5. ]


### 5. implementation of Gaussian elimination with pivoting

In [2]:
import numpy as np

In [3]:
def Gaussian_elimination_pivoting(A, b):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # size of solution vector / the square matrix A
    n=len(b) # or   n, n = A.shape
        
  
    for i in range(n):          
         # find the index of the maximal vlaue in array A[i:n,i]
        maximum = abs(A[i,i])
        max_index = i
        for j in range(i+1,n):
            if abs(A[j,i]) > maximum :
                maximum = abs(A[j,i])               
                max_index = j   
                                       
        
        # swap two max_indexs: i and max_index[i]
        temp = b[i]
        b[i] = b[max_index]
        b[max_index] = temp
        for j in range(n):
            temp = A[i,j]
            A[i,j] = A[max_index,j]
            A[max_index,j] = temp  
            
        
        if np.abs(A[i,i])<1.e-15:
            print('A is singular!')
            return    


        # Gaussian elimination
        b[i] = b[i]/A[i,i]
        A[i,:] = A[i,:]/A[i,i]
        for j in range(i+1,n):
            temp=A[j,i] 
            b[j] = b[j]-b[i]*temp
            for k in range(i,n):
                A[j,k] = A[j,k]-A[i,k]*temp           

    
    #create a new array to store the results
    x = np.zeros(n)  # or    x=b
    
    x[n-1] = b[n-1]
    for i in range(2,n+1):
        x[n-i] = b[n-i]
        for j in range(n-i+1, n):
            x[n-i] = x[n-i] - A[n-i,j]*x[j]
        
    return x

### 6. Test your implementation

In [4]:
A = np.array([[-10, 2, 0, 67], [-2, 50, -77, 1.e-5], [1, 7, 30, 8], [-10, -7, 0.001, 80]])
b = np.array([1, 2, 9, 0])

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = Gaussian_elimination_pivoting(A, b)
print("Gaussian elimination: ",x)
print("Residual: ", np.matmul(A,x)-b)

Solution by numpy solver: [0.9244595  0.31826746 0.15668124 0.14340388]
Gaussian elimination:  [0.9244595  0.31826746 0.15668124 0.14340388]
Residual:  [ 0.00000000e+00 -1.77635684e-15 -1.77635684e-15  0.00000000e+00]


In [5]:
n=20
A = np.random.rand(n, n)
b = np.random.rand(n)

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = Gaussian_elimination_pivoting(A, b)
print("Gaussian elimination: ",x)
print("Residual: ", np.matmul(A,x)-b)

Solution by numpy solver: [-1.71140894  0.7355303   1.20933082 -0.464393    0.36761446  0.67563232
  0.23702449  0.27497783 -0.02954729  0.40942985 -0.44522672 -0.12967958
 -0.71826008 -0.79501335  0.0452075   0.36954798  0.00283714  1.01739716
 -1.11238942  1.07959752]
Gaussian elimination:  [-1.71140894  0.7355303   1.20933082 -0.464393    0.36761446  0.67563232
  0.23702449  0.27497783 -0.02954729  0.40942985 -0.44522672 -0.12967958
 -0.71826008 -0.79501335  0.0452075   0.36954798  0.00283714  1.01739716
 -1.11238942  1.07959752]
Residual:  [-3.88578059e-16  6.66133815e-16  1.11022302e-16 -6.66133815e-16
 -7.77156117e-16 -2.22044605e-16 -4.44089210e-16  1.11022302e-16
  3.33066907e-16  2.22044605e-16  0.00000000e+00 -1.11022302e-16
  0.00000000e+00  6.66133815e-16  5.55111512e-16 -3.33066907e-16
  3.33066907e-16 -7.77156117e-16 -8.88178420e-16 -3.33066907e-16]


## Part c: Extension

### 5. implement Gauss-Seidel iteration for spare matrix

using the following stopping criterion
$$
\frac{\|\vec{b} - A\vec{x} \|}{\|\vec{b}\|} < 10^{-6}.
$$

In [10]:
from numpy import linalg as LA

def Gauss_Seidel_iteration_sparse(A_real, I_row, I_col, b, tol):
    # To ensure that arrays are stored in double precision.
    A_real = A_real.astype(np.float64)
    b = b.astype(np.float64)

    nonzero = len(A_real)
    n=len(b) # dimensions of the linear system of equations   
    D_diag=np.zeros(n)    # diagonal of matrix A   

    n_diag = 0
    for k in range(nonzero):
        if (I_row[k] == I_col[k]): 
            n_diag = n_diag + 1
            D_diag[I_row[k]] = A_real[k]   

    if n_diag < n :
        print('Diagonal element (%f %f)is zero!' % (i,i))
        return
        

    P_real=np.zeros(nonzero)    # matrix P = D^{-1}(L+U)
    i = 0
    for k in range(nonzero):
        if (I_row[k] > i): i = i + 1  
        P_real[k] = A_real[k] / D_diag[i]
        if (I_row[k] == I_col[k]): P_real[k] = 0.0

        
    p=np.zeros(n)   # vector p = D^{-1}b 
    for i in range(n):
        p[i]=b[i]/D_diag[i] 
   
            
    x = np.zeros(n)   # create a new array to store the results, initialised as zero
    Ax = np.zeros(n)  # Ax = A*x
    
    res = 2.*tol
    max_it = 10000
    it = 0
    while (res > tol and it < max_it):
        it = it + 1
        i = 0
        for k in range(nonzero):
            if I_row[k] == i:
                x[i] = p[i]
                i = i + 1
            x[I_row[k]] = x[I_row[k]] - P_real[k] * x[I_col[k]]

        Ax = np.zeros(n)
        for k in range(nonzero):
            Ax[I_row[k]] = Ax[I_row[k]] + A_real[k] * x[I_col[k]]  
            
        res = LA.norm(Ax-b, 2)/LA.norm(b, 2)
        print ("res ===", res)
        
    return x, it

### 6. test Gauss_Seidel_iteration_sparse(A_real, I_row, I_col, b, tol)

In [11]:
# Test different linear solvers starting from the above two-dimensional linear system
A = np.array([[3, 0, 1.5, 0], [2, 4, 2.2, 0], [0, 2, 6, 0], [0, 0, 4, -9]], dtype=np.float64)
b = np.array([3, -1, 9, 0], dtype=np.float64)

# numpy linear solver
x0 = np.linalg.solve(A,b)
print("Solution by numpy solver:", x0)

A_real = np.array([3, 1.5, 2,4 ,2.2, 2, 6, 4, -9], dtype=np.float64)
I_row = np.array([0, 0, 1, 1, 1, 2, 2, 3, 3], dtype=np.int32)
I_col = np.array([0, 2, 0, 1, 2, 1, 2, 2, 3], dtype=np.int32) 
tol = 1.e-6

x, it = Gauss_Seidel_iteration_sparse(A_real, I_row, I_col, b, tol)
print(f"Solution by Guass Seidel iteration: {x} after {it} iterations")

Solution by numpy solver: [ 0.02777778 -1.33333333  1.94444444  0.86419753]
res === 0.48847289508300185
res === 0.04884728950830017
res === 0.004884728950829996
res === 0.0004884728950829998
res === 4.884728950825872e-05
res === 4.884728950893008e-06
res === 4.88472895135457e-07
Solution by Guass Seidel iteration: [ 0.02777875 -1.33333275  1.94444425  0.86419744] after 7 iterations


### 7. Compare Gauss_Seidel_iteration_sparse() with Gaussian_elimination_pivoting()

Notice that there is a condition for Gauss-Seidel iteration: non zero element on the diagonal of the system matrix.

In [ ]:
n=20
A = np.random.rand(n, n) # generate uniform distributed elements over [0, 1)
b = np.random.rand(n)

# create a spare matrix
n = len(b)
A_real = np.array([], dtype=np.float64)
I_row = np.array([], dtype=np.int32)
I_col = np.array([], dtype=np.int32) 

for i in range(n):
    for j in range(n):
        if A[i,j] < 0.5 and i != j:
            A[i,j] = 0.0
        else:
            if i==j: A[i,j] = A[i,j] + 2.0          
            A_real = np.append(A_real, A[i,j])
            I_row = np.append(I_row, i)
            I_col = np.append(I_col, j)            
 
            
x, it = Guass_Seidel_iteration_sparse(A_real, I_row, I_col, b, 1.e-6)
print(f"Solution by Guass Seidel iteration: {x} after {it} iterations")

x = Gaussian_elimination_pivoting(A, b)
print("Gaussian elimination: ",x)
print("Residual: ", np.matmul(A,x)-b)